In [1]:
!pip install virtualenv
!virtualenv mlops-tfx

  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.2.0
    Uninstalling platformdirs-4.2.0:
      Successfully uninstalled platformdirs-4.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.1.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
created virtual environment CPython3.10.13.final.0-64 in 738ms
  creator CPy

In [2]:
!source mlops-tfx/bin/activate

In [3]:
!pip install jupyter scikit-learn tensorflow tfx==1.14.0 flask joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of apache-beam to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of apache-beam[gcp] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow-serving-api to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of google-cloud-bigtable to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of pydantic to determine which version is compatible with other requirements. This could

In [4]:
!git clone https://github.com/ArkZ10/Amazon-Review-Classification.git

Cloning into 'Amazon-Review-Classification'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), 8.75 MiB | 8.20 MiB/s, done.


# DATA PREPARATION

In [5]:
import pandas as pd
import os
df = pd.read_csv("/kaggle/working/Amazon-Review-Classification/Dataset/Dataset.csv")

In [6]:
df = df[["Text", "Cat1"]]

In [7]:
df = df.rename(columns={"Cat1" : "Label"})

In [8]:
df = df.dropna()

In [9]:
df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(min(len(x), 500)))

In [10]:
df['Label'] = df['Label'].str.replace(' ', '_')
classes_to_keep = ['toys_games', 'beauty', 'grocery_gourmet_food', 'health_personal_care']
df = df[df['Label'].isin(classes_to_keep)]
df['Label'] = df['Label'].astype('category').cat.codes

In [11]:
DATA_PATH = 'Data'

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

df.to_csv(os.path.join(DATA_PATH, 'review.csv'), index=False)

In [12]:
df.head(2)

,Text,Label
14461,Have used L'air Du Temps for many years. This ...,0
26237,"This is a nice scent, but don't over apply. It...",0


In [13]:
df['Label'].value_counts()

0    500
1    500
2    500
3    500
Name: Label, dtype: int64

# Import Library

In [14]:
!pip install tensorflow

In [15]:
import os
import pandas as pd

import tensorflow as tf
import tensorflow_model_analysis as tfma

from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator
from tfx.components import Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
import pprint

pp = pprint.PrettyPrinter()

# PIPELINE

## 1.1 Data Ingestion (ExampleGen)

In [16]:
PIPELINE_NAME = "review-classification"
SCHEMA_PIPELINE_NAME = "review-classification-schema"

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [17]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [18]:
DATA_ROOT = 'Data'

In [19]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [20]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [21]:
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type = 'GZIP')

for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Label"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Text"
    value {
      bytes_list {
        value: "Have used L\'air Du Temps for many years. This item is not L\'air Du Temps.Unfortunately all the paperwork and packaging was tossed. I won\'t order scents again."
      }
    }
  }
}

features {
  feature {
    key: "Label"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Text"
    value {
      bytes_list {
        value: "This is a nice scent, but don\'t over apply. It can be sickeningly sweet if you wear too much of it. Mix with patchouli and the scent is much better. It balances the sweet scent."
      }
    }
  }
}




## 1.2 Data Validation (StatisticsGen)

In [22]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)


interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [23]:
interactive_context.show(statistics_gen.outputs["statistics"])

## 1.3 Data Validation (Data Scheme)

In [24]:
schema_gen = SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [25]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Label',INT,required,,-
'Text',BYTES,required,,-


## 1.4 Anomaly Detection (ExampleValidator)

In [26]:
example_validator = ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema     = schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [27]:
interactive_context.show(example_validator.outputs['anomalies'])

# Data Preprocessing

## 2.1 Transform

In [28]:
TRANSFORM_MODULE_FILE = 'Amazon_review_classification_transform.py'

In [29]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
import os
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

LABEL_KEY   = "Label"
FEATURE_KEY = "Text"

# Define the set of stopwords
stop_words = set(stopwords.words('english'))

# Renaming transformed features
def transformed_name(key):
    return key + "_xf"

# Preprocess input features into transformed features
def preprocessing_fn(inputs):
    """
    inputs:  map from feature keys to raw features
    outputs: map from feature keys to transformed features
    """

    outputs = {}

    # Convert text to lowercase
    text_lower = tf.strings.lower(inputs[FEATURE_KEY])

    # Remove stopwords
    text_without_stopwords = tf.strings.regex_replace(text_lower, '|'.join(stop_words), '')

    # Store transformed text feature
    outputs[transformed_name(FEATURE_KEY)] = text_without_stopwords

    outputs[transformed_name(LABEL_KEY)]   = tf.cast(inputs[LABEL_KEY], tf.int64)


    return outputs


Writing Amazon_review_classification_transform.py


In [30]:
transform = Transform(
    examples    = example_gen.outputs['examples'],
    schema      = schema_gen.outputs['schema'],
    module_file = os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
[05/25/24 13:37:11] ERROR    listing git files failed - pretending     ]8;id=815063;file:///opt/conda/lib/python3.10/site-packages/setuptools_scm/_file_finders/git.py\git.py]8;;\:]8;id=747983;file:///opt/conda/lib/python3.10/site-packages/setuptools_scm/_file_finders/git.py#24\24]8;;\
                             there aren't any                                   


Processing ./pipelines/review-classification/_wheels/tfx_user_code_Transform-0.0+3a9f8872ff10711537cb9de53b5e9ab55b0148a1d273fe968735ee65e81274fd-py3-none-any.whl
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Processing ./pipelines/review-classification/_wheels/tfx_user_code_Transform-0.0+3a9f8872ff10711537cb9de53b5e9ab55b0148a1d273fe968735ee65e81274fd-py3-none-any.whl
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Processing ./pipelines/review-classification/_wheels/tfx_user_code_Transform-0.0+3a9f8872ff10711537cb9de53b5e9ab55b0148a1d273fe968735ee65e81274fd-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Model Training

## 3.1 Tuner

In [31]:
TUNER_MODULE_FILE = 'Amazon_review_classification_tuner.py'

In [34]:
%%writefile {TUNER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from keras_tuner.engine import base_tuner
from typing import NamedTuple, Dict, Text, Any

LABEL_KEY   = 'Label'
FEATURE_KEY = 'Text'

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=128) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE      = 1000
SEQUENCE_LENGTH = 900

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build keras tuner model"""
    embedding_dim = hp.Int('embedding_dim', min_value=30, max_value=40, step=5)
    lstm_units    = hp.Int('lstm_units', min_value=48, max_value=96, step=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.3, max_value=0.5, step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-2])
    
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name='embedding')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(4, activation='softmax')(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.SparseCategoricalCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(learning_rate),
        metrics   = ['accuracy']
    )
    
    model.summary()
    return model

TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner),
    ('fit_kwargs', Dict[Text, Any]),
])

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor  = 'val_accuracy',
    mode     = 'max',
    verbose  = 1,
    patience = 2
)

def tuner_fn(fn_args: FnArgs) -> None:
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, 5)
    val_set   = input_fn(fn_args.eval_files[0],  tf_transform_output, 5)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )
    
    # Build the model tuner
    model_tuner = kt.Hyperband(
        hypermodel   = lambda hp: model_builder(hp),
        objective    = kt.Objective('val_accuracy', direction='max'),
        max_epochs   = 5,
        factor       = 2,
        directory    = fn_args.working_dir,
        project_name = 'amazon_review_classification'
    )
    model_tuner.oracle.max_trials = 10

    return TunerFnResult(
        tuner      = model_tuner,
        fit_kwargs = {
            'callbacks'        : [early_stop_callback],
            'x'                : train_set,
            'validation_data'  : val_set,
            'steps_per_epoch'  : fn_args.train_steps,
            'validation_steps' : fn_args.eval_steps
        }
    )
     

Overwriting Amazon_review_classification_tuner.py


In [35]:
tuner = Tuner(
    module_file     = os.path.abspath(TUNER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(tuner)

Trial 10 Complete [00h 09m 03s]
val_accuracy: 0.5434243083000183

Best val_accuracy So Far: 0.5707195997238159
Total elapsed time: 01h 05m 32s
Results summary
Results in pipelines/review-classification/.temp/7/amazon_review_classification
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 02 summary
Hyperparameters:
embedding_dim: 35
lstm_units: 96
dropout_rate: 0.4
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.5707195997238159

Trial 06 summary
Hyperparameters:
embedding_dim: 35
lstm_units: 96
dropout_rate: 0.4
learning_rate: 0.01
tuner/epochs: 3
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 02
Score: 0.5707195997238159

Trial 08 summary
Hyperparameters:
embedding_dim: 35
lstm_units: 48
dropout_rate: 0.3
learning_rate: 0.01
tuner/epochs: 3
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 03
Score: 0.5707195997238159

Trial 00 summary
Hyperparameters:
embeddin

ExecutionResult(
    component_id: Tuner
    execution_id: 7
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## 3.2 Training

In [40]:
TRAINER_MODULE_FILE = 'Amazon_review_classification_trainer.py'

In [47]:
%%writefile {TRAINER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY   = 'Label'
FEATURE_KEY = 'Text'

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=128) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence
VOCAB_SIZE      = 10000
SEQUENCE_LENGTH = 900
embedding_dim   = 35

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build keras tuner model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp['embedding_dim'], name='embedding')(x)
    x = layers.Dropout(hp['dropout_rate'])(x)
    x = layers.Bidirectional(layers.LSTM(hp['lstm_units'], return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(hp['lstm_units']))(x)
    x = layers.Dropout(hp['dropout_rate'])(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(4, activation='softmax')(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.SparseCategoricalCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(hp['learning_rate']),
        metrics   = ['accuracy']
    )
    
    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features      = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    hp      = fn_args.hyperparameters['values']
    hp['epochs'] = 10
    
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor  = 'val_binary_accuracy',
        mode     = 'max',
        verbose  = 1,
        patience = 10
    )

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor        = 'val_binary_accuracy',
        mode           = 'max',
        verbose        = 1,
        save_best_only = True
    )

    callbacks = [
        tensorboard_callback,
        early_stop_callback,
        model_checkpoint_callback
    ]
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, hp['tuner/epochs'])
    val_set   = input_fn(fn_args.eval_files,  tf_transform_output, hp['tuner/epochs'])

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )
    
    # Build the model
    model = model_builder(hp)
    
    # Train the model
    model.fit(
        x                = train_set,
        validation_data  = val_set,
        callbacks        = callbacks,
        steps_per_epoch  = fn_args.train_steps,
        validation_steps = fn_args.eval_steps,
        epochs           = hp['epochs']
    )

    signatures = {
        'serving_default': _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(
                shape = [None],
                dtype = tf.string,
                name  = 'examples'
            )
        )
    }

    model.save(
        fn_args.serving_model_dir,
        save_format = 'tf',
        signatures  = signatures
    )
     

Overwriting Amazon_review_classification_trainer.py


In [48]:
trainer = Trainer(
    module_file     = os.path.abspath(TRAINER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    hyperparameters = tuner.outputs['best_hyperparameters'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(trainer)

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
[05/25/24 17:26:27] ERROR    listing git files failed - pretending     ]8;id=136335;file:///opt/conda/lib/python3.10/site-packages/setuptools_scm/_file_finders/git.py\git.py]8;;\:]8;id=380305;file:///opt/conda/lib/python3.10/site-packages/setuptools_scm/_file_finders/git.py#24\24]8;;\
                             there aren't any                                   


Processing ./pipelines/review-classification/_wheels/tfx_user_code_Trainer-0.0+d371caf4b8617ff141779190216611c27dfba7382fa4cb42896eb40a3849c884-py3-none-any.whl
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape_3 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization_3 (Text  (None, 900)               0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 900, 35)           350000    
                                                                 
 dropout_4 (Dropout)         (None, 900, 35)           0         
                              

ExecutionResult(
    component_id: Trainer
    execution_id: 11
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Model Analysis & Validation
## 4.1 Resolver

In [49]:
model_resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model          = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 12
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## 4.2 Evaluator

In [52]:
eval_config = tfma.EvalConfig(
    model_specs   = [tfma.ModelSpec(label_key = 'Label')],
    slicing_specs = [tfma.SlicingSpec()],
    metrics_specs = [
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name = 'ExampleCount'),
            tfma.MetricConfig(class_name = 'AUC'),
            tfma.MetricConfig(class_name = 'FalsePositives'),
            tfma.MetricConfig(class_name = 'TruePositives'),
            tfma.MetricConfig(class_name = 'FalseNegatives'),
            tfma.MetricConfig(class_name = 'TrueNegatives'),
            tfma.MetricConfig(class_name = 'BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold = tfma.GenericValueThreshold(
                        lower_bound = {'value': 0.5}
                    ),
                    change_threshold = tfma.GenericChangeThreshold(
                        direction = tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute  = {'value': 0.0001}
                    )
                )
            )
        ])
    ]
)
     

In [53]:
evaluator = Evaluator(
    examples       = example_gen.outputs['examples'],
    model          = trainer.outputs['model'],
    baseline_model = model_resolver.outputs['model'],
    eval_config    = eval_config
)

interactive_context.run(evaluator)
     

ExecutionResult(
    component_id: Evaluator
    execution_id: 14
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [54]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'accuracy': …

# MODEL DEPLOYMENT

In [56]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model            = trainer.outputs['model'],
    model_blessing   = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(
        filesystem = pusher_pb2.PushDestination.Filesystem(
            base_directory = 'serving_model_dir/Amazon_review_classification')
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 16
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [60]:
!zip -r pipelines.zip pipelines/
!zip -r serving_model_dir.zip serving_model_dir/
!pip freeze > requirements.txt

  adding: pipelines/ (stored 0%)
  adding: pipelines/review-classification/ (stored 0%)
  adding: pipelines/review-classification/CsvExampleGen/ (stored 0%)
  adding: pipelines/review-classification/CsvExampleGen/examples/ (stored 0%)
  adding: pipelines/review-classification/CsvExampleGen/examples/1/ (stored 0%)
  adding: pipelines/review-classification/CsvExampleGen/examples/1/Split-train/ (stored 0%)
  adding: pipelines/review-classification/CsvExampleGen/examples/1/Split-train/data_tfrecord-00000-of-00001.gz (deflated 0%)
  adding: pipelines/review-classification/CsvExampleGen/examples/1/Split-eval/ (stored 0%)
  adding: pipelines/review-classification/CsvExampleGen/examples/1/Split-eval/data_tfrecord-00000-of-00001.gz (deflated 0%)
  adding: pipelines/review-classification/Evaluator/ (stored 0%)
  adding: pipelines/review-classification/Evaluator/blessing/ (stored 0%)
  adding: pipelines/review-classification/Evaluator/blessing/13/ (stored 0%)
  adding: pipelines/review-classifica

In [61]:
asd

NameError: name 'asd' is not defined